In [ ]:
import py_entitymatching as em
import py_stringmatching as sm
import py_stringsimjoin as ssj
from PyQt4 import QtGui

In [ ]:
filepath_A = "./TableA.csv"
filepath_B = "./TableB.csv"
filepath_C = "./TableC.csv"
filepath_E = "./TableE.csv"
filepath_F = "./TableF.csv"
filepath_G = "./TableG.csv"
# read table A; table A has 'ID' as the key attribute
A = em.read_csv_metadata(filepath_A, key='ID')
# read table B; table B has 'ID' as the key attribute
B = em.read_csv_metadata(filepath_B, key='ID')
# read table C;
C = em.read_csv_metadata(filepath_C, ltable=A, rtable=B)
E = em.read_csv_metadata(filepath_E, key='ID')
# read table B; table B has 'ID' as the key attribute
F = em.read_csv_metadata(filepath_F, key='ID')
G = em.read_csv_metadata(filepath_G, ltable=A, rtable=B)
# read table E; table E has 'ID' as the key attribute

In [ ]:
G.head(7)

In [ ]:
matching_features = em.get_features_for_matching(E, F)

In [ ]:
matching_features

In [ ]:
director_features = matching_features[20:27]
director_features

In [ ]:
H = em.extract_feature_vecs(G, feature_table=director_features, attrs_after='Match')
H.head(10)

In [ ]:
# Splitting H into training (I) and test (J)
IJ = em.split_train_test(H, train_proportion=0.666)
I = IJ['train']
J = IJ['test']

In [ ]:
naive_bayes_matcher = em.NBMatcher(name='Naive_Bayes')
linear_reg_matcher = em.LinRegMatcher(name='Linear_Regression')
logistic_reg_matcher = em.LogRegMatcher(name='Logistic_Regression')
decision_tree_matcher = em.DTMatcher(name='Decision_Tree', random_state=0)
random_forest_matcher = em.RFMatcher(name='Random_Forest', random_state=0)
svm_matcher = em.SVMMatcher(name='Support_Vector_Machine')
matchers = [naive_bayes_matcher, linear_reg_matcher, logistic_reg_matcher, decision_tree_matcher,
                            random_forest_matcher, svm_matcher]
attributes_to_be_excluded = ['_id', 'ltable_ID', 'rtable_ID', 'Match']

In [ ]:
K_precision = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match', 
                          metric='precision', k= 5, random_state=0)
K_recall = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match', 
                          metric='recall', k= 5, random_state=0)
K_f1 = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match', 
                          metric='f1', k= 5, random_state=0)

In [ ]:
K_precision['cv_stats']

In [ ]:
K_recall['cv_stats']

In [ ]:
K_f1['cv_stats']

In [ ]:
K_precision['selected_matcher']

In [ ]:
K_recall['selected_matcher']

In [ ]:
K_f1['selected_matcher']

In [ ]:
L = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match',
                      metric='precision', k= 10, random_state=0)

In [ ]:
L['cv_stats']

In [ ]:
L['selected_matcher']

In [ ]:
M = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match'
                      , metric='precision', k= 20, random_state=0)

In [ ]:
M['cv_stats']

In [ ]:
M['selected_matcher']

In [ ]:
N = em.select_matcher(matchers, table=I, exclude_attrs=attributes_to_be_excluded,target_attr='Match',
                      metric='precision', k= 50, random_state=0)

In [ ]:
N['cv_stats']

In [ ]:
N['selected_matcher']

In [ ]:
UV = em.split_train_test(I, train_proportion=0.6, random_state=0)
U = UV['train']
V = UV['test']
for matcher in matchers:
    matcher.fit(table=U, exclude_attrs=attributes_to_be_excluded, target_attr='Match')
    P = matcher.predict(table=V, exclude_attrs=attributes_to_be_excluded, append=True, 
                        target_attr='Predicted_Value', inplace=False)
    predicted_result = em.eval_matches(P, 'Match', 'Predicted_Value')
    print "\n", " -------------------- [DEBUGGING]", matcher.name, " -------------------- "
    em.print_eval_summary(predicted_result)
    matcher.fit(table=I, exclude_attrs=attributes_to_be_excluded, target_attr='Match')
    P = matcher.predict(table=J, exclude_attrs=attributes_to_be_excluded, append=True, 
                        target_attr='Predicted_Value', inplace=False)
    predicted_result = em.eval_matches(P, 'Match', 'Predicted_Value')
    print "\n", " --------------------[CROSS-VALIDATION]", matcher.name, " -------------------- "
    em.print_eval_summary(predicted_result)

In [ ]:
#Debugging
#em.vis_debug_dt(decision_tree_matcher, U, V, exclude_attrs=attributes_to_be_excluded, target_attr='Match')

In [ ]:

for matcher in matchers:
    matcher.fit(table=I, exclude_attrs=attributes_to_be_excluded, target_attr='Match')
    P = matcher.predict(table=J, exclude_attrs=attributes_to_be_excluded, append=True, 
                        target_attr='Predicted_Value', inplace=False)
    predicted_result = em.eval_matches(P, 'Match', 'Predicted_Value')
    print "\n", " -------------------- ", matcher.name, " -------------------- "
    em.print_eval_summary(predicted_result)